In [5]:
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/url_spam_classification.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [48]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import pickle

In [8]:
df = pd.read_csv(file_path)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148303 entries, 0 to 148302
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   url      148303 non-null  object
 1   is_spam  148303 non-null  bool  
dtypes: bool(1), object(1)
memory usage: 1.3+ MB


In [10]:
df.head()

,url,is_spam
0,https://briefingday.us8.list-manage.com/unsubs...,True
1,https://www.hvper.com/,True
2,https://briefingday.com/m/v4n3i4f3,True
3,https://briefingday.com/n/20200618/m#commentform,False
4,https://briefingday.com/fan,True


In [11]:
label_encoder = LabelEncoder()
df['is_spam'] = label_encoder.fit_transform(df['is_spam'])

In [12]:
df.head()

,url,is_spam
0,https://briefingday.us8.list-manage.com/unsubs...,1
1,https://www.hvper.com/,1
2,https://briefingday.com/m/v4n3i4f3,1
3,https://briefingday.com/n/20200618/m#commentform,0
4,https://briefingday.com/fan,1


In [21]:
df.isnull().sum()

url        0
is_spam    0
dtype: int64

In [22]:
df.shape

(148303, 2)

In [47]:
# Create a pipeline that includes vectorization and scaling
preprocessor = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english', lowercase=True, max_df=0.7)),
    ('scaling', StandardScaler(with_mean=False))
])

In [29]:
X=df['url']
y=df['is_spam']

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)

In [38]:
# Preprocess the data
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

In [39]:
# Create the BaggingClassifier
model = BaggingClassifier()

In [42]:
# Train the model
model.fit(X_train_processed, y_train)

BaggingClassifier()

In [43]:
y_pred = model.predict(X_test_processed)

In [46]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9187884345668357


In [49]:
with open('preprocessor.pkl','wb') as file:
  pickle.dump(preprocessor,file)

In [50]:
with open('url_model.pkl','wb') as file:
  pickle.dump(model,file)